# Welcome on Swarmpy_tsp Onboarding

<a href="https://www.python.org/downloads/release/python-3100/" 
target="_blank"><img src="https://img.shields.io/badge/python-3.10-blue.svg" alt="Python Version" /></a>

___

First let's import the librairies we will use

In [1]:
import os 
import numpy as np
import pandas as pd
import plotly.express as px
import copy
os.chdir('..')

from swarmpy_tsp import *



The `ant_coder` function can load the TSP test sets of the test_set folder, namely :
- berlin52 
- ch130

`G` is the graph representation of the problem. Its a dictionnary containing a uninitalized pheromones matrix, a heuristics matrix and a cost matrix

In [11]:
G, opt_score = Antcoder()

Ant colony optmization algorithms can be built in the same fashion as a Pipeline object. You only need to put the blocks in the right order.

For example, let's instantiate two different algorithms


> Careful ! The G dictionnary is modified inplace during computation. 

In [12]:
aco_with_Daemon = ACO_Iterator(
    [
        ("Planner", Planner({"alpha": 1.0, "beta": 2.0})),
        ("Sol", SolutionConstructor()),
        ("DA", DaemonActions()),
        ("Updater", BestTourPheromonesUpdater()),
 ]
)

aco_without_Daemon = ACO_Iterator(
    [
        ("Planner", Planner({"alpha": 1.0, "beta": 2.0})),
        ("Sol", SolutionConstructor()),
        ("Updater", BestSoFarPheromonesUpdater(k=1)),
 ]
)

In [13]:
G1 = copy.deepcopy(G)

solutions_w_daemon = aco_with_Daemon.run(G=G, iter_max=20)
scores_w_daemon = np.array([el[1] for el in solutions_w_daemon]) - opt_score

solutions_wo_daemon = aco_without_Daemon.run(G=G1, iter_max=20)
scores_wo_daemon = np.array([el[1] for el in solutions_wo_daemon]) - opt_score

SwarmPy | Best score : 7735.798385217676: 100%|██████████| 20/20 [00:04<00:00,  4.90it/s]
SwarmPy | Best score : 8600.01266113827: 100%|██████████| 20/20 [00:03<00:00,  5.25it/s] 


In [14]:
scores_DA_df = pd.DataFrame({'Regret with Daemon':scores_w_daemon,'Regret without Daemon':scores_wo_daemon })
px.line(scores_DA_df,y=['Regret with Daemon', 'Regret without Daemon'], x = np.arange(len(scores_w_daemon)))